In [ ]:
from pathlib import Path
from scipy.signal import find_peaks
from scipy.stats import norm
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.patches import Rectangle
from astropy.modeling import models, fitting
import h5py
import copy
import pickle

# Project modules
from lib.Stokes import Stokes
from functions.plot_data import plot_data
from functions.plot_angle_gradient import plot_angle_gradient

In [ ]:
# Open data files
from functions.load_pickles import load_pickles
stokes_list, _ = load_pickles(select="stokes")

In [ ]:
# Extract each Stokes parameter into dictionary, to make it easier to work with
I = stokes_list['I']
Q = stokes_list['Q']
U = stokes_list['U']
V = stokes_list['V']

In [ ]:
lambda0  = [6301.51, 6302.50]   # Angstroms to cm

In [ ]:
# Fit a Gaussian to the data and find its minimum
verbose = 0

shape = np.shape(I.data_n[:,:,:2])
fit_quality = np.zeros(shape)
min_x_array = np.zeros(shape)
dl = np.zeros(shape)

for i in range(0, shape[0]):
    if np.mod(i,10) == 0:
        print(f'Row {i} of {np.shape(I.data_n)[0]}')
    for j in range(0, shape[1]):
        x = I.wave_array[:60]
        y = I.data_n[i,j,:60]
        # Normalize y for fitting stability
        y_norm = (y - np.median(y)) / np.abs(y).max()
        # Initial guess: amplitude, mean, stddev
        amplitude_guess = y_norm.min()
        mean_guess = x[np.argmin(y_norm)]
        stddev_guess = (x.max() - x.min()) / 8
        init = models.Gaussian1D(amplitude=amplitude_guess, mean=mean_guess, stddev=stddev_guess)
        fitter = fitting.LevMarLSQFitter()
        fit = fitter(init, x, y_norm)
        min_x = fit.mean.value
        min_x_array[i, j, 0] = min_x
        dl[i, j, 0] = min_x - lambda0[0]
        min_y = fit(min_x) * np.abs(y).max() + np.median(y)  # convert back to original scale
        # Fit quality: normalized RMSE
        residual = y - (fit(x) * np.abs(y).max() + np.median(y))
        rmse = np.sqrt(np.mean(residual**2))
        norm_rmse = rmse / (np.abs(y).max() - np.abs(y).min() + 1e-8)
        fit_quality[i, j, 0] = norm_rmse
        if (verbose) or (fit_quality[i,j,0] > 0.2):
            print(f"Minimum of Gaussian fit: x = {min_x}, fit quality = {norm_rmse:.4f}")
            print(f"Distance: {dl[i, j, 0]}")
            plt.plot(x, y, label='Data')
            plt.plot(x, fit(x) * np.abs(y).max() + np.median(y), label='Gaussian fit', color='orange')
            plt.axvline(min_x, color='green', linestyle='--', label='Fit minimum')
            plt.axvline(lambda0[0], color='purple', linestyle='--', label='Reference line')
            plt.legend()
            plt.title(f'Pixel ({i},{j})')
            plt.show()

In [ ]:
plot_data(dl[:,:,0], title=r"$\Delta \lambda $, in $\mathrm{\AA}$", colourmap='bwr', scale=[-0.08, 0.08])

In [ ]:
plot_data(fit_quality[:,:,0], title="RMSW", colourmap='gist_earth')

In [ ]:
c = 2.99792458e5  # [km · s−1]

# Rebecca Centeno, equation 3
v = c * dl[:,:,0] / lambda0[0]

plot_data(v, title=f"Velocity [km/s]", colourmap='bwr', scale=[-3, 3])

In [ ]:
plate_scale_x = 0.14857 # arcseconds per pixel
plate_scale_y = 0.16 # arcseconds per pixel

# Select region of quiet sun for calibration intensity calculation
xmin = 440
xmax = I.data.shape[1]
ymin = 0
ymax = 168
xwidth = xmax - xmin
ywidth = ymax - ymin
# We want to select a region with little total polarization, since this implies a low magnetic field -> quiet sun

# Plot polarization
fig, ax, img = plot_data(I.data_n[:,:,0], colourmap='magma', title="Quiet sun region on I map")

# Create a rectangle patch and add the patch to the Axes
ax.add_patch(Rectangle((xmin*plate_scale_x, ymin*plate_scale_y), xwidth*plate_scale_x, ywidth*plate_scale_y, linewidth=2, edgecolor='r', facecolor='none'))


# Save figure to file
fig.savefig("generated/polarisation_selection_square.png", dpi=150)
print("Saved figure to file", "generated/polarisation_selection_square.png")

In [ ]:
# Calulate mean velocity of quiet sun area
v_quiet_mean = v[ymin:ymax,xmin:xmax].mean()
print(f'Mean velocity in quiet sun: {v_quiet_mean} km/s')

# Calulate mean of fit
fit_quiet_mean = fit_quality[ymin:ymax,xmin:xmax].mean()
print(f'Mean fit quality in quiet sun: {fit_quiet_mean} RMSE')

In [ ]:
plot_data(v-v_quiet_mean, title=f"Velocity (corrected with quiet sun) [km/s]", colourmap='bwr', scale=[-3, 3])

In [ ]:
delta_from_mean = v_quiet_mean / c * lambda0[0]
print(f'Delta from mean: {delta_from_mean} Angstrom')

In [ ]:
fit_quality_g = fit_quality[:,:,0] * (fit_quality[:,:,0]>np.abs(delta_from_mean))
plot_data(fit_quality_g, title="RMSW", colourmap='gist_earth')

In [ ]:
x_indices = [123, 320, 422]
y_indices = [272, 373, 685]
fig, axs = plt.subplots(3, 3, figsize=(20, 16))

for row, i in enumerate(x_indices):
    for col, j in enumerate(y_indices):
        x = I.wave_array[:60]
        y = I.data_n[j, i, :60]
        y_norm = (y - np.median(y)) / np.abs(y).max()
        amplitude_guess = y_norm.min()
        mean_guess = x[np.argmin(y_norm)]
        stddev_guess = (x.max() - x.min()) / 8
        init = models.Gaussian1D(amplitude=amplitude_guess, mean=mean_guess, stddev=stddev_guess)
        fitter = fitting.LevMarLSQFitter()
        fit = fitter(init, x, y_norm)
        min_x = fit.mean.value
        min_y = fit(min_x) * np.abs(y).max() + np.median(y)
        residual = y - (fit(x) * np.abs(y).max() + np.median(y))
        rmse = np.sqrt(np.mean(residual**2))
        norm_rmse = rmse / (np.abs(y).max() - np.abs(y).min() + 1e-8)

        ax = axs[col, row]
        ax.plot(x, y, label='I data')
        ax.plot(x, fit(x) * np.abs(y).max() + np.median(y), label='Gaussian fit', color='orange')
        ax.axvline(min_x, color='green', linestyle='-.', label='Fit minimum')
        ax.axvline(lambda0[0], color='purple', linestyle='--', label='Reference line')
        ax.axvline(lambda0[0] + delta_from_mean, color='black', linestyle=':', label='Delta V quiet sun')
        # ax.axvspan(lambda0[0] + delta_from_mean, lambda0[0], color='green', alpha=0.2, label='Delta V quiet sun')
        ax.set_xlim([6301.3, 6301.7])
        ax.set_title(f'Pixel ({i},{j})\nFit min: {min_x:.4f}, RMSE: {norm_rmse:.4f}\nV: {v[j,i]-v_quiet_mean}')
        ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Plot v
fig, ax, img = plot_data(v-v_quiet_mean, title=f"Velocity (corrected with quiet sun) [km/s]", colourmap='bwr', scale=[-3, 3])

for i in range(3):
    for j in range(3):
        ax.scatter(x_indices[i]*plate_scale_x, y_indices[j]*plate_scale_y, facecolors='none', edgecolors='red')


# Save figure to file
# fig.savefig("generated/polarisation_selection_square.png", dpi=150)
# print("Saved figure to file", "generated/polarisation_selection_square.png")

In [ ]:
# Save min_x_array, fit_quality, and dl as pickle files
import pickle
with open('generated/objects/min_x_array.pickle', 'wb') as f:
    pickle.dump(min_x_array, f)
with open('generated/objects/fit_quality.pickle', 'wb') as f:
    pickle.dump(fit_quality, f)
with open('generated/objects/dl.pickle', 'wb') as f:
    pickle.dump(dl, f)
with open('generated/objects/velocity.pickle', 'wb') as f:
    pickle.dump(v, f)
print('Saved min_x_array, fit_quality, velocity, and dl to generated/ as pickle files')
